# Cutting Stock Problem

In [ ]:
from gurobipy import *
import math

## Reading an instance

We will read in an instance of the cutting stock problem just as we did for the previous workshops. The file CSP.txt contains the instance we will be looking at today. The first line of the file represents the number of customers or different roll widths (11), and the width of the stock material (1000). The first number in each subsequent line represents the width of the roll demanded and the second number is the demand of that width.

In [ ]:
def read(inputfile):
        f = open(inputfile, 'r')
        line = f.readline()
        fields = str.split(line)
        
        m = int(fields[0])
        W = int(fields[1])
        width = []
        demand = []
        for line in f:
            fields = line.split(' ')
            s = int(fields[0])
            d = int(fields[1])
            width.append(s)
            demand.append(d)

        f.close
        return m, W, width, demand

In [ ]:
m, W, width, demand =  read("../dat/CSP.txt")

## Restricted Master Problem

We initial our restricted master problem we start with the simple cutting patterns that for each roll width demanded, we will have a pattern that only cuts that roll width.

$$
\begin{align*}
		\min \quad &\sum_{i = 1}^n x_i\\
		\text{s.t.} \quad& \Big\lfloor\dfrac{W}{w_j}\Big\rfloor x_j \geq q_j \quad j = 1, \dots, m\\
		&x_i \in \mathbb{Z}_+ \quad i = 1, \dots, n
	\end{align*}
$$
### Accessing the value of the dual variables

For this problem we need to obtain the value of the dual variables associated with each constraint after solving each RMP. In previous workshops we have not paid much attention to how we save our constraints. We will create a dictionary ```orders``` which will contain our constraints. In addition we will initialize the list ```pi``` which will contain the values of the dual variables at each iteration


Since we are going to solve the restricted master problem and subproblem many times throughout the CG algorithm we will tell Gurobi to turn off the model output by setting the parameters
```python 
master.setParam("OutputFlag", 0)
subproblem.setParam("OutputFlag", 0)
```
We will provide our own printing functions that will provide a better idea of the status of the column generation algorithm than the output Gurobi gives us.

In [ ]:
master = Model('Cutting-Stock')
n = m
x = {}
for i in range(n):
    x[i] = master.addVar(vtype=GRB.CONTINUOUS, obj = 1, name="x_{}".format(i))
master.setParam("OutputFlag", 0)
master.update()
master.modelSense = GRB.MINIMIZE

orders = {} # Place Constraints in dictionary
pi = [] # Dual Values
for j in range(m):
    pi.append(0)
    orders[j] = master.addConstr(math.floor(W/float(width[j]))*x[j]  >= demand[j])

## Knapsack Subproblem

\begin{align*}
		\max \quad &\sum_{j=1}^m \bar{\pi}_j y_j\\
		\text{s.t.} \quad &\sum_{j=1}^m w_j y_j \leq W\\
		&y_j \in \mathbb{Z}_+ \quad j = 1, \dots, m
	\end{align*}

In [ ]:
subproblem = Model("Knapsack")
y = {}
for j in range(m):
    y[j] = subproblem.addVar(vtype=GRB.INTEGER, name="y_{}".format(j))

subproblem.setParam("OutputFlag", 0)
subproblem.modelSense = GRB.MAXIMIZE
subproblem.addConstr(quicksum(width[j]*y[j] for j in range(m)) <= W)
subproblem.update()

## Helper Functions for Printing

In [ ]:
def printMasterSol(master, x, n, m):
    print("-----------")
    print("Iteration: {}".format(n - m))
    print("-----------")
    print("Rolls used: {}".format(master.objval))
    for i in range(n):
        if x[i].X > 0:
            print("{} = {}".format(x[i].VarName, x[i].x))
    print("-----------")

def printDualSol(subproblem, y, m):
    print("New column found with reduced cost {}".format(1 - subproblem.objval))
    for j in range(m):
        if y[j].X > 0:
            print("{} rolls of item {}".format(y[j].X, j))

## Solve the RMP

In [ ]:
master.optimize()
printMasterSol(master, x, n, m)

## Set the objective function for the subproblem and solve

In [ ]:
for j in range(m):
    y[j].Obj = orders[j].Pi

subproblem.optimize()

## Column Generation

Until we have that $z^{SP} \leq 1$, we repeat this process of pricing a new variable, adding it to the master problem, solving the RMP and then the subproblem again.

In [ ]:
while(subproblem.objval > 1):
    printDualSol(subproblem, y, m)
    
    x[n] = master.addVar(vtype=GRB.CONTINUOUS, obj = 1, name="x_{}".format(n))
    master.update()
    
    for i in range(m):
        if y[i].x > 0.1:
            master.chgCoeff(orders[i], x[n], y[i].x)
    
    master.optimize()
    n += 1
    printMasterSol(master, x, n, m)
    
    for i in range(m):
        y[i].Obj = orders[i].Pi
    
    subproblem.optimize()

## Solve the IP

Once we have achieved the optimal LP solution, we set our $\mathbf{x}$ variables to integer and solve the IP. Since we are now interested in the status of the branch and bound procedure of our IP we will tell Gurobi to show us the output by setting the "OutputFlag" parameter to 1.

In [ ]:
master.setParam("OutputFlag", 1)
for i in range(n):
    x[i].vtype = GRB.INTEGER
master.update()
master.optimize()